In [ ]:
%%bash
# Installs the latest dev build of TVM from PyPI, with CUDA enabled. To use this,
# you must request a Google Colab instance with a GPU by going to Runtime ->
# Change runtime type -> Hardware accelerator -> GPU. If you wish to build from
# source, see see https://tvm.apache.org/docs/install/from_source.html
pip install tlcpack-nightly-cu113 --pre -f https://tlcpack.ai/wheels

Auto-scheduling a Neural Network for NVIDIA GPU
===============================================

**Author**: [Lianmin Zheng](https://github.com/merrymercy)

Auto-tuning for specific devices and workloads is critical for getting
the best performance. This is a tutorial on how to tune a whole neural
network for NVIDIA GPU with the auto-scheduler.

To auto-tune a neural network, we partition the network into small
subgraphs and tune them independently. Each subgraph is treated as one
search task. A task scheduler slices the time and dynamically allocates
time resources to these tasks. The task scheduler predicts the impact of
each task on the end-to-end execution time and prioritizes the one that
can reduce the execution time the most.

For each subgraph, we use the compute declaration in
`tvm/python/topi`{.sourceCode} to get the computational DAG in the
tensor expression form. We then use the auto-scheduler to construct a
search space of this DAG and search for good schedules (low-level
optimizations).

Different from the template-based
`autotvm <tutorials-autotvm-sec>`{.interpreted-text role="ref"} which
relies on manual templates to define the search space, the
auto-scheduler does not require any schedule templates. In other words,
the auto-scheduler only uses the compute declarations in
`tvm/python/topi`{.sourceCode} and does not use existing schedule
templates.

Note that this tutorial will not run on Windows or recent versions of
macOS. To get it to run, you will need to wrap the body of this tutorial
in a `if
__name__ == "__main__":`{.sourceCode} block.


In [ ]:
import numpy as np

import tvm
from tvm import relay, auto_scheduler
import tvm.relay.testing
from tvm.contrib import graph_executor

Define a Network
================

First, we need to define the network with relay frontend API. We can
load some pre-defined network from `tvm.relay.testing`{.sourceCode}. We
can also load models from MXNet, ONNX, PyTorch, and TensorFlow (see
`front end tutorials<tutorial-frontend>`{.interpreted-text role="ref"}).

For convolutional neural networks, although auto-scheduler can work
correctly with any layout, we found the best performance is typically
achieved with NHWC layout. We also implemented more optimizations for
NHWC layout with the auto-scheduler. So it is recommended to convert
your models to NHWC layout to use the auto-scheduler. You can use
`ConvertLayout <convert-layout-usage>`{.interpreted-text role="ref"}
pass to do the layout conversion in TVM.


In [ ]:
def get_network(name, batch_size, layout="NHWC", dtype="float32"):
    """Get the symbol definition and random weight of a network"""

    # auto-scheduler prefers NHWC layout
    if layout == "NHWC":
        image_shape = (224, 224, 3)
    elif layout == "NCHW":
        image_shape = (3, 224, 224)
    else:
        raise ValueError("Invalid layout: " + layout)

    input_shape = (batch_size,) + image_shape
    output_shape = (batch_size, 1000)

    if name.startswith("resnet-"):
        n_layer = int(name.split("-")[1])
        mod, params = relay.testing.resnet.get_workload(
            num_layers=n_layer,
            batch_size=batch_size,
            layout=layout,
            dtype=dtype,
            image_shape=image_shape,
        )
    elif name.startswith("resnet3d-"):
        n_layer = int(name.split("-")[1])
        mod, params = relay.testing.resnet.get_workload(
            num_layers=n_layer,
            batch_size=batch_size,
            layout=layout,
            dtype=dtype,
            image_shape=image_shape,
        )
    elif name == "mobilenet":
        mod, params = relay.testing.mobilenet.get_workload(
            batch_size=batch_size, layout=layout, dtype=dtype, image_shape=image_shape
        )
    elif name == "squeezenet_v1.1":
        assert layout == "NCHW", "squeezenet_v1.1 only supports NCHW layout"
        mod, params = relay.testing.squeezenet.get_workload(
            version="1.1",
            batch_size=batch_size,
            dtype=dtype,
            image_shape=image_shape,
        )
    elif name == "inception_v3":
        input_shape = (batch_size, 3, 299, 299) if layout == "NCHW" else (batch_size, 299, 299, 3)
        mod, params = relay.testing.inception_v3.get_workload(batch_size=batch_size, dtype=dtype)
    elif name == "mxnet":
        # an example for mxnet model
        from mxnet.gluon.model_zoo.vision import get_model

        assert layout == "NCHW"

        block = get_model("resnet18_v1", pretrained=True)
        mod, params = relay.frontend.from_mxnet(block, shape={"data": input_shape}, dtype=dtype)
        net = mod["main"]
        net = relay.Function(
            net.params, relay.nn.softmax(net.body), None, net.type_params, net.attrs
        )
        mod = tvm.IRModule.from_expr(net)

    return mod, params, input_shape, output_shape


# Define the neural network and compilation target
network = "resnet-18"
batch_size = 1
layout = "NHWC"
target = tvm.target.Target("cuda")
dtype = "float32"
log_file = "%s-%s-B%d-%s.json" % (network, layout, batch_size, target.kind.name)

Extract Search Tasks
====================

Next, we extract the search tasks and their weights from a network. The
weight of a task is the number of appearances of the task\'s subgraph in
the whole network. By using the weight, we can approximate the
end-to-end latency of the network as
`sum(latency[t] * weight[t])`{.sourceCode}, where
`latency[t]`{.sourceCode} is the latency of a task and
`weight[t]`{.sourceCode} is the weight of the task. The task scheduler
will just optimize this objective.


In [ ]:
# Extract tasks from the network
print("Extract tasks...")
mod, params, input_shape, output_shape = get_network(network, batch_size, layout, dtype=dtype)
tasks, task_weights = auto_scheduler.extract_tasks(mod["main"], params, target)

for idx, task in enumerate(tasks):
    print("========== Task %d  (workload key: %s) ==========" % (idx, task.workload_key))
    print(task.compute_dag)

Begin Tuning
============

Now, we set some options for tuning and launch the search tasks

-   `measure_ctx`{.sourceCode} launches a different process for
    measurement to provide isolation. It can protect the main process
    from GPU crashes during measurement and avoid other runtime
    conflicts.
-   `min_repeat_ms`{.sourceCode} defines the minimum duration of one
    \"repeat\" in every measurement. This can warmup the GPU, which is
    necessary to get accurate measurement results. Typically, we
    recommend a value \>= 300 ms.
-   `num_measure_trials`{.sourceCode} is the number of measurement
    trials we can use during the tuning. You can set it to a small
    number (e.g., 200) for a fast demonstrative run. In practice, we
    recommend setting it around `900 * len(tasks)`{.sourceCode}, which
    is typically enough for the search to converge. For example, there
    are 24 tasks in resnet-18, so we can set it as 20000. You can adjust
    this parameter according to your time budget.
-   In addition, we use `RecordToFile`{.sourceCode} to dump measurement
    records into a log file, The measurement records can be used to
    query the history best, resume the search, and do more analyses
    later.
-   see `auto_scheduler.TuningOptions`{.interpreted-text role="any"},
    `auto_scheduler.LocalRPCMeasureContext`{.interpreted-text
    role="any"} for more parameters.


In [ ]:
def run_tuning():
    print("Begin tuning...")
    measure_ctx = auto_scheduler.LocalRPCMeasureContext(repeat=1, min_repeat_ms=300, timeout=10)

    tuner = auto_scheduler.TaskScheduler(tasks, task_weights)
    tune_option = auto_scheduler.TuningOptions(
        num_measure_trials=200,  # change this to 20000 to achieve the best performance
        runner=measure_ctx.runner,
        measure_callbacks=[auto_scheduler.RecordToFile(log_file)],
    )

    tuner.tune(tune_option)


# We do not run the tuning in our webpage server since it takes too long.
# Uncomment the following line to run it by yourself.

# run_tuning()

::: {.note}
::: {.admonition-title}
Note
:::

Explain the printed information during tuning

During the tuning, a lot of information will be printed on the console.
They are used for debugging purposes. The most important info is the
output of the task scheduler. The following table is a sample output.

``` {.sourceCode .c}
----------------------------------------------------------------------
------------------------------  [ Task Scheduler ]
----------------------------------------------------------------------
|  ID  | Latency (ms) | Speed (GFLOPS) | Trials |
-------------------------------------------------
|    0 |        0.005 |           0.88 |     64 |
|    1 |        0.010 |          99.10 |     64 |
|    2 |        0.006 |           0.00 |     64 |
|    3 |        0.145 |         979.78 |    384 |
|    4 |        0.130 |        1097.02 |    384 |
|    5 |        0.143 |         992.69 |    384 |
|    6 |        0.076 |        1526.86 |    192 |
|    7 |        0.115 |         999.44 |    320 |
|    8 |        0.079 |        1449.39 |    320 |
|    9 |        0.122 |         938.73 |    384 |
|   10 |        0.063 |        1832.98 |    192 |
|   11 |        0.072 |        1763.62 |    256 |
|   12 |        0.062 |        2036.40 |    192 |
|   13 |        0.068 |        1874.44 |    192 |
|   14 |        0.049 |        2346.50 |    128 |
|   15 |        0.076 |        1694.31 |    256 |
|   16 |        0.067 |        1933.30 |    448 |
|   17 |        0.076 |        1680.90 |    256 |
|   18 |        0.022 |          98.43 |     64 |
|   19 |        0.076 |        3112.55 |    192 |
|   20 |        0.013 |        2026.44 |     64 |
|   21 |        0.011 |        1136.69 |     64 |
|   22 |        0.013 |         992.47 |     64 |
|   23 |        0.020 |         627.56 |     64 |
-------------------------------------------------
Estimated total latency: 1.587 ms  Trials: 4992  Used time : 13296 s  Next ID: 3
```

This table lists the latency and (estimated) speed of all tasks. It also
lists the allocation of measurement trials for all tasks. The last line
prints the total weighted latency of these tasks, which can be a rough
estimation of the end-to-end execution time of the network. The last
line also prints the total number of measurement trials, total time
spent on auto-tuning and the id of the next task to tune.

There will also be some \"tvm::Error\"s and CUDA errors, because the
auto-scheduler will try some invalid schedules. You can safely ignore
them if the tuning can continue, because these errors are isolated from
the main process.
:::


::: {.note}
::: {.admonition-title}
Note
:::

Terminate the tuning earlier

You can terminate the tuning earlier by forcibly killing this process.
As long as you get at least one valid schedule for each task in the log
file, you should be able to do the compilation (the secion below).
:::


Compile and Evaluate
====================

After auto-tuning, we can compile the network with the best schedules we
found. All measurement records are dumped into the log file during
auto-tuning, so we can read the log file and load the best schedules.


In [ ]:
# Compile with the history best
print("Compile...")
with auto_scheduler.ApplyHistoryBest(log_file):
    with tvm.transform.PassContext(opt_level=3, config={"relay.backend.use_auto_scheduler": True}):
        lib = relay.build(mod, target=target, params=params)

# Create graph executor
dev = tvm.device(str(target), 0)
module = graph_executor.GraphModule(lib["default"](dev))
data_tvm = tvm.nd.array((np.random.uniform(size=input_shape)).astype(dtype))
module.set_input("data", data_tvm)

# Evaluate
print("Evaluate inference time cost...")
print(module.benchmark(dev, repeat=3, min_repeat_ms=500))

Other Tips
==========

1.  During the tuning, the auto-scheduler needs to compile many programs
    and extract feature from them. This part is CPU-intensive, so a
    high-performance CPU with many cores is recommended for faster
    search.
2.  You can use
    `python3 -m tvm.auto_scheduler.measure_record --mode distill -i log.json`{.sourceCode}
    to distill the large log file and only save the best useful records.
3.  You can resume a search from the previous log file. You just need to
    add a new argument `load_log_file`{.sourceCode} when creating the
    task scheduler in function `run_tuning`{.sourceCode}. Say,
    `tuner = auto_scheduler.TaskScheduler(tasks, task_weights, load_log_file=log_file)`{.sourceCode}
4.  If you have multiple target GPUs, you can use all of them for
    measurements to parallelize the measurements. Check this
    `section <tutorials-autotvm-scale-up-rpc-tracker>`{.interpreted-text
    role="ref"} to learn how to use the RPC Tracker and RPC Server. To
    use the RPC Tracker in auto-scheduler, replace the runner in
    `TuningOptions`{.sourceCode} with
    `auto_scheduler.RPCRunner`{.interpreted-text role="any"}.
